In [1]:
import numpy as np
import preprocess_mesh as pm
import pyvista as pv
import uac_boundaries as ub

In [2]:
patient_id = "01"
infile_name = f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk"
anatomical_tags = {"LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

vtk_mesh = pv.read(infile_name)
triangular_mesh = pm.convert_unstructured_to_polydata_mesh(vtk_mesh)

In [3]:
inner_boundaries, inner_boundary_global_inds = ub.get_feature_inner_boundaries(
    triangular_mesh, anatomical_tags.values()
)
lipv_lspv_path, ripv_rspv_path, lipv_ripv_path, lspv_rspv_path = ub.compute_roof_frame(
    triangular_mesh, anatomical_tags, inner_boundary_global_inds
)

In [4]:
lipv_boundary = inner_boundaries[1]
lipv_start = lipv_boundary.find_closest_point(lipv_lspv_path.points[0])
lipv_end = lipv_boundary.find_closest_point(lipv_ripv_path.points[0])
lipv_path_one = ub.construct_path_between_boundary_points(
    lipv_boundary, lipv_start, lipv_end, direction=[0, 1]
)
lipv_path_two = ub.construct_path_between_boundary_points(
    lipv_boundary, lipv_start, lipv_end, direction=[1, 0]
)

lspv_boundary = inner_boundaries[2]
lspv_start = lspv_boundary.find_closest_point(lipv_lspv_path.points[1])
lspv_end = lspv_boundary.find_closest_point(lspv_rspv_path.points[0])
lspv_path_one = ub.construct_path_between_boundary_points(
    lspv_boundary, lspv_start, lspv_end, direction=[0, 1]
)
lspv_path_two = ub.construct_path_between_boundary_points(
    lspv_boundary, lspv_start, lspv_end, direction=[1, 0]
)

ripv_boundary = inner_boundaries[3]
ripv_start = ripv_boundary.find_closest_point(lipv_ripv_path.points[1])
ripv_end = ripv_boundary.find_closest_point(ripv_rspv_path.points[0])
ripv_path_one = ub.construct_path_between_boundary_points(
    ripv_boundary, ripv_start, ripv_end, direction=[0, 1]
)
ripv_path_two = ub.construct_path_between_boundary_points(
    ripv_boundary, ripv_start, ripv_end, direction=[1, 0]
)

rspv_boundary = inner_boundaries[4]
rspv_start = rspv_boundary.find_closest_point(lspv_rspv_path.points[1])
rspv_end = rspv_boundary.find_closest_point(ripv_rspv_path.points[1])
rspv_path_one = ub.construct_path_between_boundary_points(
    rspv_boundary, rspv_start, rspv_end, direction=[0, 1]
)
rspv_path_two = ub.construct_path_between_boundary_points(
    rspv_boundary, rspv_start, rspv_end, direction=[1, 0]
)

In [ ]:
plotter = pv.Plotter(window_size=[900, 900])
for path in lipv_lspv_path, ripv_rspv_path, lipv_ripv_path, lspv_rspv_path:
    plotter.add_mesh(path, color="yellow", line_width=5, render_lines_as_tubes=True)
plotter.add_mesh(triangular_mesh, color="lightgray", show_edges=True, edge_opacity=0.3)
for inner_boundary in inner_boundaries:
    plotter.add_mesh(inner_boundary, color="blue", line_width=5, render_lines_as_tubes=True)
plotter.add_points(
    lipv_boundary.points[lipv_path_one], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    lipv_boundary.points[lipv_path_two], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    lspv_boundary.points[lspv_path_one], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    lspv_boundary.points[lspv_path_two], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    ripv_boundary.points[ripv_path_one], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    ripv_boundary.points[ripv_path_two], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    rspv_boundary.points[rspv_path_one], color="red", point_size=10, render_points_as_spheres=True
)
plotter.add_points(
    rspv_boundary.points[rspv_path_two], color="red", point_size=10, render_points_as_spheres=True
)
plotter.show()

Widget(value='<iframe src="http://localhost:39213/index.html?ui=P_0x7f43fc64f4d0_2&reconnect=auto" class="pyvi…